In [1]:
import numpy as np
from keras.callbacks import LambdaCallback
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.models import Sequential
from keras.optimizers import RMSprop
import nltk
from nltk.corpus import gutenberg




C:\Users\Vikas\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
files= ['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt']
raw = gutenberg.raw(files)


listOfchars = sorted(list(set(raw)))
print('total number of chars:', len(listOfchars))
character_to_Index = dict((c, i) for i, c in enumerate(listOfchars))
Index_to_character = dict((i, c) for i, c in enumerate(listOfchars))

max_length_of_seq = 50+1

sentences = []
next_chars = []

for i in range(0, len(listOfchars) - max_length_of_seq):
    sentences.append(listOfchars[i: i + max_length_of_seq])
    next_chars.append(listOfchars[i + max_length_of_seq])

x = np.zeros((len(sentences), max_length_of_seq, len(listOfchars)), dtype=np.bool)
y = np.zeros((len(sentences), len(listOfchars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, character_to_Index[char]] = 1
    y[i, character_to_Index[next_chars[i]]] = 1




total number of chars: 82


In [5]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_length_of_seq, len(listOfchars))))
model.add(Dense(len(listOfchars)))
model.add(Activation('softmax'))


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

def on_epoch_end(epoch, logs):
    model_json = model.to_json()
    with open("modelc.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("modelc.h5")


model.fit(x, y, batch_size=128, epochs=100,callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])